<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/S10_online_fulfillment/S10_Module1_Online_Fulfillment_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for Session 10: Online fulfillment analytics

---
In this session, we also begin by installing the (i) Pyomo package and (ii) the linear programming solver GLPK (GNU Linear Programming Kit). Please feel free to revisit [Session 9](https://github.com/acedesci/scanalytics/blob/master/S8_9_retail_analytics/S9_Module2A_Retail_Price_Optimization.ipynb) for further information. 

In [ ]:
# Install Pyomo and GLPK
!pip install -q pyomo
!apt-get install -y -qq glpk-utils #if GLPK is used

# Blocks 1&2: Data input and parameters

We have two different datasets, but their structures are the same. The dataset 1 is the same as in the paper whereas the dataset 2 is a larger dataset to demonstrate how data can be adapted.

1.   **Fulfillment center (FC) data**: each dataset contains a set of fulfillment centers $FC$ (*listFC*), each of which has a given level of inventory ${\bf X}$ (*inventoryFC*). Customers can place an order for either a single item or multiple items, but if no fulfillment center has all the items available, the retailer has to resort to split deliveries, which obviously affect the shipping expenses. Therefore, our optimization model has to take account of the probability that the fulfillment center has 'other items in order' (*probMultiAvailability*).
2.   **Customer region data**: each customer zone (*listRegion*) has to be associated with a certain demand over the next $\tau$ days (*demandValue*) for customer $j$ and the proportion of orders that are for multiple items (*probMultiItem*). 
2.   **Shipment cost data**: *costSingle$[i,j]$* denotes the cost of a single delivery from Fulfillment Center (FC) *i* to Customer Zone *j* or $c_{ij}$. Please note that for Data 1, since there is only one customer zone (Kansas), there is only one cost from each origin (i.e., from FC *i* to Kansas). *avgNoMultiItem* stands for the *average number of items in a multi-item order*. This will help us compute the average shipping cost per item for multi-item orders. More specifically $\omega = 1/\mbox{avgNoMultiItem}$. 









In [ ]:
# Data 1 (same as in the paper)

# Fulfillment center (FC) data
listFC = ['Utah', 'Nevada']
inventoryFC = dict(zip(listFC, [5, 20]))
print(inventoryFC)
probMultiAvailability = dict(zip(listFC, [0.5, 0.2]))
print(probMultiAvailability)

# Customer region data
listRegion = ['Kansas']
demandValue = dict(zip(listRegion, [2*10]))
print(demandValue)
probMultiItem = dict(zip(listRegion, [0.75]))
print(probMultiItem)

# Shipment cost data
costSingle = [[9], [12]]
print(costSingle)

# average number of items in a multi-item order
avgNoMultiItem = 3.0
# we can calculate the multi-item shipping discount $\omega$
shippingDiscount = 1/avgNoMultiItem

# Prepare shipment cost above in dictionary format
costSingleDict = {}
for i in range(len(listFC)):
  for j in range(len(listRegion)):
    costSingleDict[(listFC[i],listRegion[j])] = costSingle[i][j]

print(costSingleDict)  

In [ ]:
# Data 2

# Fulfillment center (FC) data
listFC = ['Delta-BC', 'Brampton-ON', 'Ottawa-ON']
inventoryFC = dict(zip(listFC, [37, 85, 25]))
print(inventoryFC)
probMultiAvailability = dict(zip(listFC, [0.32, 0.45, 0.17]))
print(probMultiAvailability)

# Customer region data
listRegion = ['Toronto', 'Montreal', 'Calgary', 'Vancouver']
demandValue = dict(zip(listRegion, [45, 27, 15, 33]))
print(demandValue)
probMultiItem = dict(zip(listRegion, [0.73, 0.68, 0.54, 0.64]))
print(probMultiItem)

# Shipment cost data
costSingle = [[24.5, 25.5, 18.1, 12.3],
        [13.6, 17.5, 22.8, 23.6],
        [18.1, 14.1, 21.1, 22.8]]

# average number of items in a multi-item order
avgNoMultiItem = 2.5
# we can calculate the multi-item shipping discount $\omega$
shippingDiscount = 1/avgNoMultiItem

# Prepare shipment cost above in dictionary format
costSingleDict = {}
for i in range(len(listFC)):
  for j in range(len(listRegion)):
    costSingleDict[(listFC[i],listRegion[j])] = costSingle[i][j]

print(costSingleDict)  

# Block 3: Create an optimization model

### Block 3.1: Variable declarations

As we all learned in [Session 9](https://github.com/acedesci/scanalytics/blob/master/S8_9_retail_analytics/S9_Module2A_Retail_Price_Optimization.ipynb), an optimization model consists of (i) decision variables, (ii) objective function, and (iii) constraints.

Our decision variables include model.x, model.y and model.w, which are all nonnegative, denoting the flow from FC *i* to customer region *j*. In particular,

model.**x**$[i, j]$ (or $x_{ij}$) denotes the decision variable for **unsplit** flow from FC *i* to **multi-item** customer *j*. The unit cost of this flow is $\omega c_{ij}$;

model.**y**$[i, j]$ (or $y_{ij}$) denotes the decision variable for **split** flow from FC *i* to **multi-item** customer *j*. The unit cost of this flow is $2\omega c_{ij}$ (two shipments are used if the order cannot be shipped in one shipment);

model.**w**$[i, j]$ (or $w_{ij}$) denotes the decision variable for flow from FC *i* to **single-item** customer *j*.  The unit cost of this flow is $c_{ij}$.

On the last line, we also create an object to store the shadow prices (dual variables). This is used if we don't want to solve the LP multiple times.

In [ ]:
from pyomo.environ import *

model = ConcreteModel()
# Variables
model.x = Var(listFC, listRegion, within = NonNegativeReals)
model.y = Var(listFC, listRegion, within = NonNegativeReals)
model.w = Var(listFC, listRegion, within = NonNegativeReals)

model.inventoryOnHand = ConstraintList()
model.demandSingle = ConstraintList()
model.demandMulitiple = ConstraintList()
model.maxMultiShipment = ConstraintList()

# create an object to access to shadow prices. Please note that the codes must be exactly written as 
# model.dual = Suffix(direction=Suffix.IMPORT_EXPORT) in order for it to work because it was hard coded in pyomo.
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

### Block 3.2: Objective function

For notational simplication, let

$c_{ij}=$ costSingle$[i,j]$, the cost of a single delivery from Fulfillment Center (FC) *i* to Customer Zone *j*;

$\omega=1$ / avgNoMultiItem, expected discount of sending multi-item order in one package.

Then, we have the general form of the objective function as follows

$$\min\limits_{x,y,w} \left\{ \sum_{i\in\mathcal{FC}} \sum_{j\in\mathcal{J}}\left(  c_{ij}\cdot w_{ij} + \omega \cdot c_{ij}\cdot x_{ij} + 2\omega \cdot c_{ij}\cdot y_{ij} \right) \right\}$$

In [ ]:
# Objective function

obj_expr = sum(costSingleDict[(i,j)]*model.w[(i,j)] + shippingDiscount*costSingleDict[(i,j)]*model.x[(i,j)] \
               + (2*shippingDiscount)*costSingleDict[(i,j)]*model.y[(i,j)] for i in listFC for j in listRegion)
print(obj_expr)
model.OBJ = Objective(expr = obj_expr, sense = minimize)

### Block 3.3: Constraints functions

**Constraint Set 1**: Inventory availability at FC *i*

This set of constraints ensures that the total demand (orders) assigned to FC *i* is less than or equal to the inventory at FC *i*.

Let $X_i$ denote the inventory level at FC *i* and $\mathcal{FC}$ denote the list of all FCs.
$$\sum_{j\in\mathcal{J}} \left( w_{ij} + x_{ij} + y_{ij} \right) \leq X_i, \forall i\in \mathcal{FC} $$

In [ ]:
# Constraints 1 Inventory availability at FC i

for i in listFC:
  const_expr = sum(model.w[(i,j)] + model.x[(i,j)] + model.y[(i,j)] for j in listRegion) <= inventoryFC[i] 
  print(const_expr)
  model.inventoryOnHand.add(expr = const_expr)


**Constraint Set 2**: Future demand for single-item order in region *j*

This set of constraints ensures that the single-item demand at Customer (Demand) Zone *j* is satisfied.

Let $D_j$ denote the demand at Customer Zone *j*, $\mathcal{J}$ denote the list of all Customer (Demand) Zones and $\lambda_j$ denote the proportion of orders that are for multiple items (probMultiItem) $\Rightarrow\left(1-\lambda_j\right)$ equals the proportion of orders that are for a single item.
$$\sum_{i \in \mathcal{FC}} w_{ij} = D_j \cdot \left( 1-\lambda_j \right), \forall j\in \mathcal{J} $$

In [ ]:
# Constraints 2 Future demand for single-item order in region j 

for j in listRegion:
  const_expr = sum(model.w[(i,j)] for i in listFC) == demandValue[j]*(1-probMultiItem[j])
  print(const_expr)
  model.demandSingle.add(expr = const_expr)

**Constraint Set 3**: Future demand for multi-item order in region *j*

This set of constraints ensures that the multi-item demand at Customer (Demand) Zone *j* is satisfied.

Using the similar notations as in *Constraint set 2*, we attain
$$\sum_{i \in \mathcal{FC}}  \left( x_{ij} + y_{ij} \right) = D_j \cdot \lambda_j, \forall j\in \mathcal{J} $$

In [ ]:
# Constraints 3 Future demand for multi-item order in region j 

for j in listRegion:
  const_expr = sum(model.x[(i,j)] + model.y[(i,j)] for i in listFC) == demandValue[j]*probMultiItem[j]
  print(const_expr)
  model.demandMulitiple.add(expr = const_expr)

**Constraint Set 4**: Estimated maximum number multi-item shipments from FC *i* to customer region *j*

It should be noted that unsplit delivery for a multi-item order from FC *i* to Customer Region *j* is effected only when FC *i* has 'other items in order'. Let $\rho_i$ denote the probability that FC *i* has 'other items in order' (probMultiAvailability). Then, we impose the following constraints. 

$$ x_{ij} \leq D_j \cdot \lambda_j \cdot \rho_i, \forall i\in\mathcal{FC}, j\in \mathcal{J} $$

In [ ]:
# Constraints #4 Estimated maximum number multi-item shipments from FC i to customer regions j 

for i in listFC:
  for j in listRegion:
    const_expr = model.x[(i,j)] <= demandValue[j]*probMultiItem[j]*probMultiAvailability[i]
    print(const_expr)
    model.maxMultiShipment.add(expr = const_expr)

You can print the entire model to check if you want

In [ ]:
# This is commented so that we won't print out unless necessary. You can outcomment to print it.
# model.pprint()

# Block 4: Solution and results

Finally, we call for the solver and obtain the solution. The first line indicates which solver we want to use and the second line solves the model (this is equivalent to *fit()* in sklearn). The last line *displays* the solution.

In [ ]:
# Solve the model
opt = SolverFactory('glpk')
opt.solve(model) 

model.display()

In order to quickly determine the approximate cost-to-go without solving multiple times the LP model above, we can also make use of the shadow price. The codes below print out the shadow prices of all the constraints. Only the shadow prices of the constraint *inventoryOnHand* are used to calculate the approximate cost-to-go, i.e., $$C_{k}({\bf X_k})=\min_{i\in \mathcal{FC}}\left({c_{ik}+C_{k+1}({\bf X_{k+1}})}\right)=\min_{i\in \mathcal{FC}}\left({c_{ik}+C_{k+1}({\bf X_{k}})-\pi_{i}}\right).$$

In [ ]:
# Obtain reduced cost for each constraint
model.dual.display()